In [3]:
#top 10 trending movies + year + cast
import imdb
ia = imdb.IMDb()
trending = ia.get_popular100_movies()

#bolds the title
print( '\033[1m' + 'Top 10 Trending Movies\n'+'\033[0m') 

count = 1 
for i in range(10): 
    print(count,'.',trending[i])
    print('Year: ',trending[i]['year'])
    name = trending[i]   
    name = str(trending[i])
    search = ia.search_movie(name)    
    id=trending[i].movieID        
    movie = ia.get_movie(id)
    cast = movie['cast']
    actor0 = cast[0]
    actor1 = cast[1]
    actor2 = cast[2]
    print('Starring: ',actor0,",",actor1,",",actor2)
    print('===================')
    count+=1




Top 10 Trending Movies

1 . The Batman
Year:  2022
Starring:  Robert Pattinson , Zoë Kravitz , Jeffrey Wright
2 . The Northman
Year:  2022
Starring:  Alexander Skarsgård , Nicole Kidman , Claes Bang
3 . Thor: Love and Thunder
Year:  2022
Starring:  Taika Waititi , Karen Gillan , Natalie Portman
4 . Fantastic Beasts: The Secrets of Dumbledore
Year:  2022
Starring:  Jude Law , Cara Mahoney , Mads Mikkelsen
5 . Everything Everywhere All at Once
Year:  2022
Starring:  Michelle Yeoh , Stephanie Hsu , Ke Huy Quan
6 . X
Year:  2022
Starring:  Mia Goth , Jenna Ortega , Brittany Snow
7 . The Unbearable Weight of Massive Talent
Year:  2022
Starring:  Nicolas Cage , Pedro Pascal , Tiffany Haddish
8 . K.G.F: Chapter 2
Year:  2022
Starring:  Yash , Sanjay Dutt , Srinidhi Shetty
9 . Death on the Nile
Year:  2022
Starring:  Michael Rouse , Alaa Safi , Orlando Seale
10 . Choose or Die
Year:  2022
Starring:  Iola Evans , Asa Butterfield , Robert Englund


In [2]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
service = input('What streaming service are you using? (AmazonPrime, DisneyPlus, Hulu, Netflix) \n')

if service == 'AmazonPrime':
    data = pd.read_csv(r'C:\Users\Jason\Documents\CIS 9590\Data\amazon_prime_titles.csv')
elif service == 'DisneyPlus':
    data = pd.read_csv(r'C:\Users\Jason\Documents\CIS 9590\Data\disney_plus_titles.csv')
elif service == 'Hulu':
    data = pd.read_csv(r'C:\Users\Jason\Documents\CIS 9590\Data\hulu_titles.csv')
elif service == 'Netflix':
    data = pd.read_csv(r'C:\Users\Jason\Documents\CIS 9590\Data\netflix_titles.csv')
else:
    print('Please select a proper streaming service.')

data = data.dropna(subset=['country','rating'])

movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration'], axis=1)
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration'], axis=1)

directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

binary = pd.concat([binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)

actors2 = []

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

binary2 = pd.concat([binary_countries2, binary_genres2], axis=1, ignore_index=True)

def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")
    
title = input('Pick a movie/tv show you would like a reccomendation for. \n')
recommender(title)

What streaming service are you using? (AmazonPrime, DisneyPlus, Hulu, Netflix) 
Netflix
Pick a movie/tv show you would like a reccomendation for. 
Jaws


,title,director,cast,country,rating,listed_in,description,cos_sim
5005,Schindler's List,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,R,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...,0.925820
4938,Rocky,John G. Avildsen,"Sylvester Stallone, Talia Shire, Burt Young, C...",United States,PG,"Action & Adventure, Classic Movies, Dramas",Sylvester Stallone shot to fame as Rocky Balbo...,0.857143
5147,Superfly,Gordon Parks,"Ron O'Neal, Carl Lee, Sheila Frazier, Julius H...",United States,R,"Action & Adventure, Classic Movies, Dramas","To get away from the thug life, cocaine dealer...",0.857143
3818,Bonnie and Clyde,Arthur Penn,"Warren Beatty, Faye Dunaway, Michael J. Pollar...",United States,R,"Action & Adventure, Classic Movies, Dramas","Bonnie Parker and Clyde Barrow are young, in l...",0.857143
4352,Indiana Jones and the Last Crusade,Steven Spielberg,"Harrison Ford, Sean Connery, Denholm Elliott, ...",United States,PG-13,"Action & Adventure, Children & Family Movies, ...","Accompanied by his father, Indiana Jones sets ...",0.857143
